## 순환 신경망 구현 및 학습
- tensorflow 를 이용하여 순환 신경망 구현할 것! 

In [1]:
import tensorflow as tf

In [3]:
 tf.__version__

'2.2.0'

## 하이퍼 파라미터

In [4]:
EPOCHS = 10
NUM_WORDS = 10000 # 분석을 하기위해서 만개의 단어만 사용하겠다는 의미

## 모델정의

In [6]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16) 
        # 원핫 벡터가 10000일때, 첫번째가 MY 였다면, 특정 단어들이 원핫 벡터들이 하나씩 표현됨. 
        # Embedding : 원핫 벡터는 정수값이라서, 심지어 0,1의 바이너리값이여서, 실수값을 가져오기 위해서 길이가 10000인 원핫 벡터를 길이가 16인 피쳐 벡터로 바꿔주는 역할을 함
        
        self.rnn = tf.keras.layers.SimpleRNN(32) # rnn layer를 사용하는데, 심플하게 바릴라 rnn사용
        self.dense = tf.keras.layers.Dense(2, activation = 'softmax') # 감정분석 을 할거기 때문에, 길이가 2인 소프트 맥스를 사용 
        
    def call(x , training = None, mask = None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## 학습, 테스트 루프 정의

In [7]:
# Implement training loop 

@tf.function
def train_step(model, inputs , labels, loss_object, optimizer, train_loss, train_accuracy ) :
    with tf.GradientTape() as tape :
        predictions = model(inputs, training = True )
        loss = loss_object(labels, predictions )
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions )
    

# Implement algorithm test
@tf.function 
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs, training = False)
    
    t_loss = loss_object(lavels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터 셋 준비

In [9]:
imdb = tf.keras.datasets.imdb
(x_trian, y_train), (x_test, y_test) = imdb.load_data(num_words = NUM_WORDS)


# 데이터의 L 의 길이가 다르기 때문에, PRE -PADDING 을 해준다. 
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, 
                                                        value = 0,  # 0으로 패팅을 해줌
                                                        padding = 'pre',
                                                        maxlen = 32) # 최대길이를 32로 잘라주면서 앞에 패팅 0해줌

x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, 
                                                        value = 0,  # 0으로 패팅을 해줌
                                                        padding = 'pre',
                                                        maxlen = 32) # 최대길이를 32로 잘라주면서 앞에 패팅 0해줌

# 학습할 때마다 셔플이 일어날 수 있도록 설정 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_dataset = tf.data

10043392/17464789 [================>.............] - ETA: 1s

KeyboardInterrupt: 